In [2]:
from vllm import AsyncEngineArgs, AsyncLLMEngine
from dotenv import load_dotenv
import os

mistral = "/home/yuri011228/ai2-server/models/mistral-7b"

engine_args = AsyncEngineArgs(
    model=mistral,
    tensor_parallel_size=1, # GPU 개수
    gpu_memory_utilization=0.95,
    max_num_seqs = 100, # 동시에 받을 수 있는 요청 개수
    max_model_len=4096, # input + output 토큰 길이
    max_num_batched_tokens=8192) 

llm = AsyncLLMEngine.from_engine_args(engine_args)

load_dotenv()

# 확인용
# os.environ["LANGSMITH_TRACING"]  
# print(os.getenv("QDRANT_HOST"))
# print(os.getenv("QDRANT_PORT"))

INFO 05-12 00:54:30 [config.py:689] This model supports multiple tasks: {'reward', 'score', 'classify', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 05-12 00:54:30 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-12 00:54:31 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/home/yuri011228/ai2-server/models/mistral-7b', speculative_config=None, tokenizer='/home/yuri011228/ai2-server/models/mistral-7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(sho

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:01<00:02,  1.30s/it]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:02<00:01,  1.40s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:04<00:00,  1.49s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:04<00:00,  1.46s/it]



INFO 05-12 00:54:37 [loader.py:458] Loading weights took 4.68 seconds
INFO 05-12 00:54:38 [gpu_model_runner.py:1291] Model loading took 13.4967 GiB and 4.870459 seconds
INFO 05-12 00:54:48 [backends.py:416] Using cache directory: /home/yuri011228/.cache/vllm/torch_compile_cache/8f3f217165/rank_0_0 for vLLM's torch.compile
INFO 05-12 00:54:48 [backends.py:426] Dynamo bytecode transform time: 10.70 s
INFO 05-12 00:54:49 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 05-12 00:54:57 [monitor.py:33] torch.compile takes 10.70 s in total
INFO 05-12 00:54:59 [kv_cache_utils.py:634] GPU KV cache size: 50,320 tokens
INFO 05-12 00:54:59 [kv_cache_utils.py:637] Maximum concurrency for 4,096 tokens per request: 12.29x
INFO 05-12 00:55:32 [gpu_model_runner.py:1626] Graph capturing finished in 32 secs, took 0.51 GiB
INFO 05-12 00:55:32 [core.py:163] init engine (profile, create kv cache, warmup model) took 54.21 seconds
INFO 05-12 00:55:32 [core_client.py:435] C

True

In [3]:
dummy = {
  "email": "ConconDev",
  "date": "2024-09-06",
  "level": 1,
  "title": "알람 구독 서비스 개선: SseEmitter 활용 및 사용자별 Emitter 관리",
  "keywords": [
    "SSE",
    "SseEmitter",
    "알람 구독",
    "사용자별 관리"
  ],
  "til": "# 알람 구독 서비스 개선: SseEmitter 활용 및 사용자별 Emitter 관리\n\n## 1. 오늘 배운 내용\n\n오늘 저는 알람 구독 서비스를 개선하기 위해 `SseEmitter`를 활용하고, 각 사용자별로 `SseEmitter`를 저장하고 관리하는 기능을 구현했습니다.  `SseEmitter`는 서버에서 클라이언트로 실시간 데이터를 스트리밍하는 데 유용한 API입니다.\n\n## 2. 개념 정리\n\n*   **SSE (Server-Sent Events):** 서버에서 클라이언트로 단방향 통신을 가능하게 하는 웹 기술입니다. 서버가 새로운 이벤트 발생 시 클라이언트에게 자동으로 업데이트를 전송합니다.\n*   **SseEmitter:** SSE 통신을 위한 객체입니다.  데이터를 발행하거나, 연결을 종료하거나, 오류를 처리하는 등의 기능을 제공합니다.\n*   **ConcurrentHashMap:** 여러 스레드에서 동시에 접근해도 안전하게 데이터를 저장하고 검색할 수 있는 해시맵입니다.  여기서는 사용자 ID를 키로 하고 `SseEmitter`를 값으로 저장하는 데 사용되었습니다.\n\n## 3. 해당 개념이 필요한 이유\n\n기존 알람 구독 방식은 클라이언트가 주기적으로 서버에 요청을 보내는 방식으로, 서버 부하가 심하고 효율성이 떨어졌습니다. `SseEmitter`를 사용하면 서버는 새로운 알람이 발생했을 때만 클라이언트에 데이터를 전송하므로, 서버 자원을 절약하고 실시간성을 높일 수 있습니다. 또한, 사용자별로 `SseEmitter`를 관리함으로써, 특정 사용자에 대한 알람만 전송하도록 할 수 있어 더욱 효율적인 알람 구독 시스템을 구축할 수 있습니다.\n\n## 4. 개념을 활용하는 방법\n\n1.  `SseEmitter` 객체를 생성하고 초기화합니다.\n2.  사용자 ID를 키로, `SseEmitter` 객체를 값으로 `ConcurrentHashMap`에 저장합니다.\n3.  클라이언트에서 SSE 연결을 설정하고, 서버로부터 데이터를 수신합니다.\n4.  사용자가 알람 구독/취소 요청을 하면, 해당 사용자의 `SseEmitter`를 업데이트하거나 삭제합니다.\n\n## 5. 문제 해결 과정\n\n*   처음에는 `SseEmitter`의 생명주기를 제대로 관리하지 못하여 메모리 누수가 발생했습니다.  `SseEmitter` 객체의 `close()` 메서드를 호출하여 리소스 누수를 방지했습니다.\n*   사용자 ID 중복 문제를 해결하기 위해 사용자 ID를 키로 사용하는 `ConcurrentHashMap`을 사용했습니다.\n*   클라이언트에서 SSE 연결을 안정적으로 유지하기 위해 에러 핸들링 로직을 추가했습니다.\n\n## 6. 하루 회고\n\n오늘은 알람 구독 서비스의 성능 개선을 위해 중요한 기술인 `SseEmitter`를 익히고 적용하는 시간을 가졌습니다.  `SseEmitter`의 동작 원리를 이해하고, 실제 서비스에 적용하면서 많은 어려움을 겪었지만, 결국 성공적으로 구현할 수 있었습니다. 앞으로는 `SseEmitter`를 더 깊이 이해하고, 다양한 응용 분야에 적용해보고 싶습니다.\n\n## 7. 전체적으로 개조식 문장 구성\n\n*   **목표:** 알람 구독 서비스의 실시간성 및 효율성 향상\n*   **핵심 기술:** SSE, SseEmitter, ConcurrentHashMap\n*   **구현 단계:**\n    *   `SseEmitter` 객체 생성 및 초기화\n    *   사용자별 `SseEmitter` 저장 및 관리 (`ConcurrentHashMap`) \n    *   SSE 연결 설정 및 데이터 수신\n    *   알람 구독/취소 요청 처리\n    *   메모리 누수 방지 및 에러 핸들링\n\n"
}

prompt1 = """
당신은 사용자의 기술 학습 기록을 바탕으로, 기술 면접 질문을 생성하는 AI입니다.

조건
- 레벨: {level}
- 사용자 TIL: {til}

아래 정보를 참고하여,
[검색 결과, 참고 문서] {retrieved}

※ level에 따라 질문 수준을 조절해서 면접 질문을 작성해주세요:
- level "1": 깊은 기술 이해와 실무 경험 기반 질문
- level "2": 개념적 이해를 묻는 질문
- level "3": 기본 개념을 묻는 질문

반드시 하나의 질문만 만들고 **한국어**로 작성하세요.

"""


In [3]:
## 연습 
# from langgraph.graph import StateGraph
# from pydantic import BaseModel
# from typing import List, Optional

# class ContentState(BaseModel):
#     question: str
#     answer: str 

# class QAState(BaseModel):
#     email: str
#     date: str
#     level: int
#     title: str
#     keywords: List[str]
#     til: str

#     qa_0: Optional[ContentState] = None
#     qa_1: Optional[ContentState] = None
#     qa_2: Optional[ContentState] = None
#     summary: Optional[str] = None
#     content: Optional[List[ContentState]] = None

# # dummy retriever
# async def retriever_node(state: QAState) -> dict:
#     return {}

# # 질문 노드
# async def question0_node(state: QAState): return {}
# async def question1_node(state: QAState): return {}
# async def question2_node(state: QAState): return {}

# # 답변 노드
# async def answer0_node(state: QAState): return {"qa_0": {"question": "Q0", "answer": "A0"}}
# async def answer1_node(state: QAState): return {"qa_1": {"question": "Q1", "answer": "A1"}}
# async def answer2_node(state: QAState): return {"qa_2": {"question": "Q2", "answer": "A2"}}

# # 집계 노드: 세 답변이 모두 도달할 때까지 기다림
# async def join_answers_node(state: QAState) -> dict:
#     return {}

# # 요약 노드
# async def summary_node(state: QAState): return {"summary": "요약", "content": []}

# # 그래프 구성
# workflow = StateGraph(QAState)

# # 노드 정의
# workflow.add_node("retriever", retriever_node)
# workflow.add_node("question0", question0_node)
# workflow.add_node("question1", question1_node)
# workflow.add_node("question2", question2_node)
# workflow.add_node("answer0", answer0_node)
# workflow.add_node("answer1", answer1_node)
# workflow.add_node("answer2", answer2_node)
# workflow.add_node("join_answers", join_answers_node)
# workflow.add_node("summary_node", summary_node)



# workflow.add_edge("join_answers", "summary_node")
# workflow.set_finish_point("summary_node")

# graph = workflow.compile()


In [4]:
from langgraph.graph import StateGraph
from pydantic import BaseModel
from typing import List, Optional, Dict, Any

class ContentState(BaseModel):
    question: str
    answer: str 

class QAState(BaseModel):
    email: str
    date: str
    level: int
    title: str
    keywords: List[str]
    til: str

    retrieved_texts: Optional[List[str]] = None
    similarity_score: Optional[float] = None

    question: Optional[str] = None
    answer: Optional[str] = None

    #output
    summary: Optional[str] = None
    content: Optional[List[ContentState]] = None

qa_input = QAState(
    email=dummy['email'],
    date=dummy['date'],
    level=dummy['level'],
    title=dummy['title'],
    keywords=dummy['keywords'],
    til=dummy['til']
)


In [34]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from uuid import uuid4
from vllm import SamplingParams
import os


qdrant = QdrantClient(
    host=os.getenv("QDRANT_HOST"), 
    port=os.getenv("QDRANT_PORT"))
embedding_model = SentenceTransformer("BAAI/bge-m3", device="cpu")

def embed_text(text: str) -> list[float]:
    return embedding_model.encode(text).tolist()

async def retriever_node(state: QAState) -> dict:
    query = state.title + " " + " ".join(state.keywords)
    query_vector = embed_text(query)

    collection_names = ["ai", "cloud", "frontend", "backend"]
    best_score = 0.0
    retrieved_texts: List[str] = []

    for col in collection_names:
        results = qdrant.search(
            collection_name=col,
            query_vector=query_vector,
            limit=3,
            with_payload=True
        )

        if results and results[0].score > best_score:
            best_score = results[0].score
            retrieved_texts = [h.payload["text"] for h in results if "text" in h.payload]


    return {
        "similarity_score": best_score,
        "retrieved_texts": retrieved_texts
    }

prom = """
당신은 사용자의 기술 학습 기록을 바탕으로, 기술 면접 질문을 생성하는 AI입니다.

아래 정보를 참고하여,
[TIL 본문] {til}
[RAG 검색 결과] {text}
[선택한 난이도] {level}

※ level에 따라 질문 수준을 조절해서 면접 질문을 작성해주세요:
- level "1": 깊은 기술 이해와 실무 경험 기반 질문
- level "2": 개념적 이해를 묻는 질문
- level "3": 기본 개념을 묻는 질문

모든 질문과 답변은 반드시 **한국어**로 작성하세요.

"""
# 질문 생성 노드
async def question0_node(state: QAState) -> Dict[str, Any]:
    text = "\n\n".join(state.retrieved_texts or [])
    prompt = prom.format(
        til=state.til,
        text=text,
        level=state.level
    )

    sampling_params = SamplingParams(
        temperature=0.7,
        max_tokens=512
    )

    request_id = str(uuid4())
    final_text = ""

    async for output in llm.generate(prompt, sampling_params, request_id=request_id):
        final_text = output.outputs[0].text.strip()

    # 질문만 추출
    match = re.search(r"질문[:：]\s*(.+)", final_text)
    question = match.group(1).strip() if match else "질문 생성 실패"

    return {
        "qa_0": ContentState(question=question, answer="")
    }

async def question1_node(state: QAState) -> Dict[str, Any]:
    generated_q = f"Q1: '{state.title}' 관련 두 번째 질문입니다."
    return {"qa_1": ContentState(question=generated_q, answer="")}

async def question2_node(state: QAState) -> Dict[str, Any]:
    generated_q = f"Q2: '{state.title}' 관련 세 번째 질문입니다."
    return {"qa_2": ContentState(question=generated_q, answer="")}

# 답변 생성 노드
async def answer0_node(state: QAState) -> dict:
    return {"qa_0": {"question": state.qa_0.question, "answer": "답변0"}}


async def summary_node(state: QAState) -> dict:
    # state.qa_0, qa_1, qa_2에 담긴 질문/답변을 모아서 최종 summary, content 생성
    content = [state.qa_0]
    summary = "요약 생성 완료"
    return {
        "summary": summary,
        "content": content
    }

In [35]:
from langchain_core.runnables import RunnableLambda
workflow = StateGraph(QAState)

# 노드 추가
workflow.add_node("retriever",RunnableLambda(retriever_node).with_config({"run_name": "retriever"}))

workflow.add_node("summary_node",RunnableLambda(summary_node).with_config({"run_name": "summary"}))

# 질문/답변 노드 3세트
for i in range(3):
    q_name = f"question{i}"
    a_name = f"answer{i}"

    workflow.add_node(
        q_name,
        RunnableLambda(globals()[f"{q_name}_node"]).with_config({"run_name": q_name})
    )
    workflow.add_node(
        a_name,
        RunnableLambda(globals()[f"{a_name}_node"]).with_config({"run_name": a_name})
    )

    workflow.add_edge("retriever", q_name)
    workflow.add_edge(q_name, a_name)

# 엣지 연결
workflow.set_entry_point("retriever")

workflow.add_edge("answer0", "summary_node")
workflow.add_edge("answer1", "summary_node")
workflow.add_edge("answer2", "summary_node")

workflow.set_finish_point("summary_node")

graph = workflow.compile()


In [36]:
graph


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [54]:
# 그래프 정의 및 구성
from langgraph.graph import END
from langchain_core.runnables import RunnableLambda

workflow = StateGraph(QAState)

# 노드 선언
workflow.add_node("retriever", RunnableLambda(retriever_node).with_config({"run_name": "retriever"}))

# 질문 생성 노드
workflow.add_node("question0", RunnableLambda(question0_node).with_config({"run_name": "question0"}))
workflow.add_node("question1", RunnableLambda(question1_node).with_config({"run_name": "question1"}))
workflow.add_node("question2", RunnableLambda(question2_node).with_config({"run_name": "question2"}))

# 답변 생성 노드
workflow.add_node("answer0", RunnableLambda(answer0_node).with_config({"run_name": "answer0"}))
workflow.add_node("answer1", RunnableLambda(answer1_node).with_config({"run_name": "answer1"}))
workflow.add_node("answer2", RunnableLambda(answer2_node).with_config({"run_name": "answer2"}))

# fallback + summary
# workflow.add_node("fallback_generate", RunnableLambda(fallback_generate_node).with_config({"run_name": "fallback_generate"}))
workflow.add_node("summary_node", RunnableLambda(summary_node).with_config({"run_name": "summary_node"}))

# 조건부 분기 설정
# def route_by_similarity(state: QAState) -> str:
#     return "generate_seq" if state.similarity_score >= 0.5 else "fallback_generate"

# workflow.add_conditional_edges(
#     "retriever",
#     route_by_similarity,
#     {
#         "generate_seq": ["question0", "question1", "question2"],
#         "fallback_generate": "fallback_generate"
#     }
# )

workflow.add_edge("retriever", "question0")
workflow.add_edge("retriever", "question1")
workflow.add_edge("retriever", "question2")


# 각 질문 생성 → 답변 생성 연결
workflow.add_edge("question0", "answer0")
workflow.add_edge("question1", "answer1")
workflow.add_edge("question2", "answer2")

# 각 답변 노드 → summary
workflow.add_edge("answer0", "summary_node")
workflow.add_edge("answer1", "summary_node")
workflow.add_edge("answer2", "summary_node")

# fallback도 summary로
# workflow.add_edge("fallback_generate", "summary_node")

# 시작, 종료 지점 설정
workflow.set_entry_point("retriever")
workflow.set_finish_point("summary_node")


In [55]:
graph = workflow.compile()
# result = await graph.ainvoke(qa_input)

In [12]:
# # 그래프 정의 및 구성
# from langgraph.graph import END
# from langchain_core.runnables import RunnableLambda

# workflow = StateGraph(QAState)

# # 노드 선언
# workflow.add_node("retriever", 
#     RunnableLambda(retriever_node).with_config({"run_name": "retriever"}))
# workflow.add_node("generation0", 
#     RunnableLambda(generation0_node).with_config({"run_name": "generation0"}))
# workflow.add_node("generation1", 
#     RunnableLambda(generation1_node).with_config({"run_name": "generation1"}))
# workflow.add_node("generation2", 
#     RunnableLambda(generation2_node).with_config({"run_name": "generation2"}))
# workflow.add_node("fallback_generate", 
#     RunnableLambda(fallback_generate_node).with_config({"run_name": "fallback_generate"}))
# workflow.add_node("summary_node", 
#     RunnableLambda(summary_node).with_config({"run_name": "summary_node"}))

# # 시작 지점 설정
# workflow.set_entry_point("retriever")

# # 조건부 분기
# def route_by_similarity(state: QAState) -> str:
#     return "generation" if state.similarity_score >= 0.50 else "fallback_generate"

# workflow.add_conditional_edges(
#     "retriever",
#     route_by_similarity,
#     {
#         "generation": ["generation0", "generation1", "generation2"],
#         "fallback_generate": "fallback_generate"
#     })

# # 병렬 분기: generation / fallback → summary
# workflow.add_edge("generation0", "summary_node")
# workflow.add_edge("generation1", "summary_node")
# workflow.add_edge("generation2", "summary_node")
# workflow.add_edge("fallback_generate", "summary_node")

# # 종료 지점 설정
# workflow.set_finish_point("summary_node")

# graph = workflow.compile()
# result = await graph.ainvoke(qa_input)

INFO 05-08 08:49:23 [async_llm.py:228] Added request 04e11ffa-2ba6-4d30-ad82-5622e8a42637.


### 레거시

In [6]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from langgraph.graph import END
from langchain_core.runnables import RunnableLambda
from vllm import SamplingParams
from uuid import uuid4
from vllm import AsyncEngineArgs, AsyncLLMEngine


qdrant = QdrantClient(
    host=os.getenv("QDRANT_HOST"), 
    port=os.getenv("QDRANT_PORT"))

embedding_model = SentenceTransformer("BAAI/bge-m3", device="cpu")

def embed_text(text: str) -> list[float]:
    return embedding_model.encode(text).tolist()

async def retriever_node(state: QAState) -> dict:
    query = state.title + " " + " ".join(state.keywords)
    query_vector = embed_text(query)

    collection_names = ["ai", "cloud", "frontend", "backend"]
    best_score = 0.0

    for col in collection_names:
        results = qdrant.search(
            collection_name=col,
            query_vector=query_vector,
            limit=3,
            with_payload=True
        )

        if results and results[0].score > best_score:
            best_score = results[0].score
            retrieved_texts = [h.payload["text"] for h in results if "text" in h.payload]

    return {
        "similarity_score": best_score,
        "retrieved_texts": retrieved_texts
    }

result = await retriever_node(qa_input)
print(result)

async def generate_node(state: QAState) -> dict:

    retrieved = "\n\n".join(state.retrieved_texts or [])

    prompt = prompt1.format(
        til=state.til,
        level=state.level,
        retrieved=retrieved
    )

    sampling_params = SamplingParams(
        temperature=0.7,
        max_tokens=2048,
        stop_token_ids=[2], 
    )

    request_id = str(uuid4())
    final_text = ""

    async for output in llm.generate(
        prompt=prompt,
        sampling_params=sampling_params,
        request_id=request_id
    ):
        final_text = output.outputs[0].text.strip()

    return {
        "answer_raw": final_text
    }

final = await generate_node(qa_input)
print(final["answer_raw"])

# def route_by_similarity(state: QAState) -> str:
#     return "generation"

# async def generation_node(state: QAState) -> dict:
#     return {"log": "generation 노드 실행됨"}

# async def summary_node(state: QAState) -> dict:
#     return {"log_summary": "요약 생성 완료"}

# async def final_node(state: QAState) -> dict:
#     return {"log_final": "최종 결과 병합 완료"}

# workflow = StateGraph(QAState)

# # 노드 선언
# workflow.add_node("retriever", 
#     RunnableLambda(retriever_node).with_config({
#         "run_name": "retriever"}))
# workflow.add_node("generate", 
#     RunnableLambda(generate_node).with_config({
#         "run_name": "generate"}))
# workflow.add_node("summary_node", 
#     RunnableLambda(summary_node).with_config({
#         "run_name": "summary_node"}))

# # 시작 지점 설정
# workflow.set_entry_point("retriever")

# workflow.add_edge("retriever", "generate")
# workflow.add_edge("generate", "summary_node")

# # 종료 지점 설정
# workflow.set_finish_point("summary_node")

# graph = workflow.compile()
# result = await graph.ainvoke(qa_input)

{'similarity_score': 0.52782375, 'retrieved_texts': ['서버 전송 이벤트(Server-sent events, SSE)는 클라이언트가 HTTP 연결을 통해 서버로부터 자동 업데이트를 수신할 수 있도록 하는 서버 푸시 기술이며, 초기 클라이언트 연결이 설정된 후 서버가 클라이언트를 향한 데이터 전송을 시작하는 방법을 설명한다. 이는 일반적으로 브라우저 클라이언트에 메시지 업데이트 또는 지속적인 데이터 스트림을 보내는 데 사용되며 클라이언트가 이벤트 스트림을 수신하기 위해 특정 URL을 요청하는 EventSource라는 자바스크립트 API를 통해 기본 브라우저 간 스트리밍을 향상시키도록 설계되었다. EventSource API는 WHATWG에 의해 HTML Living Standard의 일부로 표준화되었다. SSE의 미디어 유형은 text/event-stream이다. 파이어폭스 6+, 구글 크롬 6+, 오페라 11.5+, 사파리 5+, 마이크로소프트 엣지 79+ 등 모든 최신 브라우저는 서버에서 전송되는 이벤트를 지원한다.', '푸시 기법 기술은 풀 기법으로 명명된 웹브라우저와 비교할 수 있다. 현재 인터넷에서 사용되고 있는 웹브라우저는 사용할 때에 사용자가 항상 자신이 갖고자 하는 정보를 소유한 서버에게 정보를 요청하며 웹브라우저의 사용과 목적지에 대한 최종결정 역시 사용자가 결정할 수 있다. 반면 푸시 기법은 사용자가 일일이 요청하지 않아도 사용자에게 자동으로 뉴스나 사용자가 원하는 특별한 정보, 예를 들면 증권시장의 주기적인 정보 같은 것을 제공한다. 이 두 가지 기법의 차이점은 정보의 흐름을 누가 통제하느냐 하는 점에 있다고 할 수 있다. 풀 기법 하에서는 사용자 (즉 소비자)들이 정보 취득 및 정보의 접촉을 마음대로 통제할 수 있으나, 푸시 기법 하에서는 정보를 전달하는 쪽에서(즉 광고주)정보의 흐름을 직접 통제할 수 있게 된다.', '푸시 기법의 가장 큰 이점은 역시 정보의 맞춤화(Customization)가 가능

/tmp/ipykernel_7213/925149334.py:27: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


### 1. 당신은 오늘 어떤 기술을 배웠습니까?

오늘은 알람 구독 서비스를 개선하기 위해 사용되는 `SseEmitter`와 관련된 기술에 대해 배웠습니다. `SseEmitter`는 서버에서 클라이언트로 실시간 데이터를 스트리밍하는 데 사용되는 WebSocket의 대안입니다.

### 2. SSE(Server-Sent Events)로 알람 구독 서비스를 개선하는 이유는 무엇입니까?

기존의 알람 구독 방식에서는 클라이언트가 주기적으로 서버에 요청을 보내며, 서버는 해당 요청에 대해 알람 정보를 반환합니다. 하지만 이 방식에는 몇 가지 문제점이 있습니다. 먼저, 서버는 모든 클라이언트의 요청을 처리해야 하므로 서버 부하가 심할 수 있습니다. 또한, 클라이언트는 주기적인 요청을 보내야 하므로 효율성도 떨어집니다. 이를 해결하기 위해, 알람 구독 서비스를 SSE와 함께 구현하여 서버는 알람이 발생할 때만 클라이언트에게 알람 정보를 전송하도록 합니다.

### 3. SSE(Server-Sent Events)는 무엇입니까?

SSE(Server-Sent Events)는 서버에서 클라이언트로 단방향 통신을 가능하게 하는 Web Technology입니다. 서버가 새로운 이벤트가 발생할 때 클라이언트에게 자동으로 업데이트를 전송합니다. 이는 일반적인 HTTP 프로토콜을 사용합니다. 이를 통해 서버는 클라이언트에 실시간 데이터를 전송할 수 있습니다.

### 4. SSE를 구현하는데 사용되는 SseEmitter는 무엇입니까?

`SseEmitter`는 SSE를 구현하는 데 사용되는 Java API입니다. 이 API는 데이터를 발행하거나, 연결을 종료하거나, 오류를 처리하는 등의 기능을 제공합니다.

### 5. SseEmitter를 사용하여 알람 구독 서비스를 구현하는 방법은 무엇입니까?

1. `SseEmitter` 객체를 생성하고 초기화합니다.
2. 사용자 ID를 키로, `SseEmitter` 객체를 값으로 `ConcurrentHashMap`에 저장합니다.
3. 클라이언

In [111]:
# # 생성 → 파싱 테스트 흐름
# result = await fallback_generate_node(qa_input)

# print("LLM 생성된 answer_raw:")
# print(result["answer_raw"])
# print("\n---------------------------\n")

# # QAState에 넣어서 파싱
# qa_input.answer_raw = result["answer_raw"]
# parsed = await parsing_node(qa_input)

# print("파싱된 결과:")
# for item in parsed["content"]:
#     print("질문", item.question)
#     print("답변", item.answer, "\n")


In [ ]:
# from dotenv import load_dotenv
# from langsmith import traceable

# load_dotenv()

# @traceable(name="generate-question")
# def generate_question(input_text: str) -> str:
#     return f"이건 입력에 대한 질문입니다: {input_text}"

# generate_question("LangGraph 연동 테스트")

'이건 입력에 대한 질문입니다: LangGraph 연동 테스트'